In [ ]:
import numpy as np
import random
from collections import defaultdict

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = [''] * 9  # Empty board
        self.current_winner = None

    def reset(self):
        self.board = [''] * 9
        self.current_winner = None
        return tuple(self.board)

    def available_actions(self):
        return [i for i, x in enumerate(self.board) if x == '']

    def make_move(self, position, player):
        if self.board[position] == '':
            self.board[position] = player
            if self.winner(player):
                self.current_winner = player
            return True
        return False

    def winner(self, player):
        # Check rows, columns, diagonals
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]  # diagonals
        ]
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False

    def is_draw(self):
        return all(self.board) and self.current_winner is None

class QLearningAgent:
    def __init__(self, name = 'X', alpha=0.1, gamma=0.5, epsilon=0.8):
        self.name = name
        self.q_table = defaultdict(float)  # Q-values
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.state = None
        self.action = None
        self.reward = 0

    def choose_action(self, state, actions):
        if random.random() < self.epsilon:
            action = random.choice(actions)
        else:
            action = self.exploit(state, actions)
        return action

    def exploit(self, state, actions):
        q_values = [self.q_table[(state, a)] for a in actions]
        max_q = max(q_values)
        action = actions[q_values.index(max_q)] 
        return action

    def update_q(self, state, actions):
        max_future_q = max([self.q_table[(state, action)] for action in actions], default=0) 
        current_q = self.q_table[(self.state, self.action)]
        self.q_table[(self.state, self.action)] = current_q + self.alpha * (self.reward + self.gamma * max_future_q - current_q)

def train(agent_x, agent_o, env, episodes=10000, decay_rate=0.995):
    for episode in range(episodes):
        state = env.reset()
        done = False
        current_player = agent_x
        agent_x.action = None
        agent_o.action = None

        while not done:
            actions = env.available_actions()
            action = current_player.choose_action(state, actions)

            env.make_move(action, current_player.name)
            next_state = tuple(env.board)
            current_player.reward = 0
            done = env.current_winner is not None or env.is_draw()

            if current_player.action is not None:
                current_player.update_q(state, actions)
            
            current_player.state = state
            current_player.action = action
                      
            state = next_state
                
            current_player = agent_o if current_player.name == 'X' else agent_x
        actions = []
        if env.current_winner is not None:
            current_player.reward = -1
            current_player.update_q(state, actions)
            winner =  agent_o if current_player.name == 'X' else agent_x
            winner.reward = 1
            winner.update_q(state, actions)
        else:
            current_player.update_q(state, actions)
        if episode%2000 == 0:
            agent_x.epsilon *= decay_rate
            agent_o.epsilon *= decay_rate
            print(len(agent_x.q_table), 'epsilon:', agent_x.epsilon)
            for i in range(9):
                print(i, agent_x.q_table[(tuple(['']*9), i)])

# Initialize environment and agents
env = TicTacToe()
agent_x = QLearningAgent()
agent_o = QLearningAgent(name='O')

train(agent_x, agent_o, env, episodes=500000)

In [ ]:
def play_with(agent_x):
    env=TicTacToe()

    agent_x.state = tuple(env.board)
    done = False
    current_player = 'X'  

    while not done:
        if current_player != agent_x.name:  # Human player
            print("Your turn. Choose a position (0-8):")
            try:
                position = int(input().strip())
                if position not in env.available_actions():
                    print("Invalid move. Try again.")
                    continue
            except ValueError:
                print("Please enter a valid number between 0 and 8.")
                continue

        else:  # AI's turn
            print("Agent's turn...")
            position = agent_x.exploit(tuple(env.board), env.available_actions())

        # Make the move and update the board
        env.make_move(position, current_player)
        print_board(env.board)

        # Check for game end conditions
        if env.current_winner:
            if current_player != agent_x.name:
                print("Congratulations! You won!")
            else:
                print("Agent wins. Better luck next time!")
            done = True
        elif env.is_draw():
            print("It's a draw!")
            done = True

        # Switch players
        current_player = 'X' if current_player == 'O' else 'O'

    print("Game over!")


def print_board(board):
    # Helper function to print the board
    for i in range(0, 9, 3):
        print('|'.join([board[i + j] if board[i + j] != '' else str(i + j) for j in range(3)]))
    print("-" * 5)
    
play_with(agent_o)